Создание собственных классов.

Хорошим тоном является создание исключения в рамках класса-наследника от обобщенного исключения Exception. И последующие исключения наследовать от него

In [1]:
class CSCException(Exception):
    pass

In [4]:
class TestFailure(CSCException):
    def __str__(self):
        return "lectute test failed"

In [5]:
try:
    do_something()
except TestFailure:
    pass

NameError: name 'do_something' is not defined

Сообщения по ошибкам можно расширять и забирать в отдельные переменные

In [6]:
error = None
try:
    do_something()
except NameError as e:
    error = e

In [7]:
error

NameError("name 'do_something' is not defined")

In [8]:
error.args

("name 'do_something' is not defined",)

In [9]:
error.__traceback__

Помимо этого ошибку можно трассировать с использованием соответствующего модуля

In [10]:
import traceback

In [11]:
traceback.print_tb(error.__traceback__)

  File "<ipython-input-6-3cfd60334ba0>", line 3, in <module>
    do_something()


В Python 3 реализовано адресное поднятие исключений через ключевое слово from.

In [12]:
try:
    {}['foo']
except KeyError as e:
    raise RuntimeError("Ooops!") from e

RuntimeError: Ooops!

### Менеджеры контекста

Два абсолютно аналогичных блока

In [ ]:
r = acquire_resource()
try:
    do_something(r)
finally:
    release_resource(r)

In [ ]:
with acquire_resource() as r:
    do_something(r)
# Действие release_resource(r) будет выполнено интерпритатором самостоятельно

Можно использовать более чем один контекстный менеджер контекста, но для их разделения ОБЯЗАТЕЛЬНО использовтаь обратный слеш

In [ ]:
with acquire_resource() as r, \
acquire_other_resource() as other:
    do_something(r, other)

In [ ]:
# Или так
with acquire_resource() as r:
    with acquire_other_resource() as other:
        do_something(r, other)

В Python имеется модуль tempfile позволяющий создавать временные файлы и дректории.

Можно создать свой собственный контекстрый менеджер. Он обязан иметь два метода: enter и exit

In [13]:
class sinchronize:
    def __init__(self):
        self.lock = threading.Lock()
        
    def __enter__(self):
        self.lock.acquire()
        
    def __exit__(self, *exc_info):
        self.lock.release()

In [ ]:
# И далее его можно бужет вызвать как контекстный менеджер
with sinchronize:
    do_something()

Достаточно полезный менеджер контекста, позволяющий в своих рамках изменять рабочую диретокрию

In [14]:
import os
class cd:
    def __init__(self, path):
        self.path = path
        
    def __enter__(self):
        self.saved_dir = os.getcwd()
        os.chdir(self.path)
        
    def __exit__(self, *exc_info):
        os.chdir(self.saved_dir)

In [15]:
print(os.getcwd())

E:\Disk.Yandex\3.Программирование\balovstvo&checking\Python_Курс Сергея Лебедева


In [16]:
with cd('E:\\Disk.Yandex\\'):
    print(os.getcwd())

E:\Disk.Yandex


+ Менеджеры контекста - досаточно удобный способ управлять жизненным циклом ресурсов в Python.
+ Для работы с менеджером контекста исползуется оператор **with**
+ Менеджером контекста является экземпляр любого класса реализующий методы \_\_enter\_\_ и \_\_exit\_\_
+ Неокторые встроенные типы, например файлы и примитывы синхронизации уже поддерживают протокол менеджеров контекста - этим лучше пользоваться.

Для менеджеров контекста имеется достаточно полезный модуль contextlib, имеющий множество фич, одной из которых является suppress, который подавляет исключения указанных типов.

In [17]:
from contextlib import suppress

In [19]:
with suppress(FileNotFoundError):
    os.remove("example.txt")